`log_softmax`函数是softmax函数的一个变体，它直接计算softmax输出的对数形式。这种形式的softmax函数在深度学习中非常有用，尤其是在涉及到对数似然函数的优化问题时。`log_softmax`函数的数学表达式如下：

给定一个实数向量 \( \mathbf{x} \)，其元素为 \( x_i \)，`log_softmax`函数计算得到的向量 \( \mathbf{y} \) 的第 \( i \) 个元素 \( y_i \) 可以通过以下公式得到：

$ y_i = x_i - \log\left(\sum_{j=1}^{K} e^{x_j}\right) $

其中，\( K \) 是向量 \( \mathbf{x} \) 的维度，即类别的数量，\( e \) 是自然对数的底数。

这个公式的含义是：对于向量 \( \mathbf{x} \) 中的每个元素 \( x_i \)，我们首先计算所有元素的指数和的对数，然后从 \( x_i \) 中减去这个值。这样做的结果是，每个 \( y_i \) 都表示为 \( x_i \) 相对于其他元素 \( x_j \) 的对数概率。

在PyTorch中，`log_softmax`函数的实现利用了广播和自动求导的特性，可以自动计算出每个元素的对数概率，而不需要显式地计算指数和。这使得计算更加高效，并且避免了数值溢出的问题。

# NUMPY

根据您提供的前向传播代码，我们可以看到网络的结构是这样的：

- 输入 `X` 的形状是 (4, 2)。
- 第一层权重 `W1` 的形状是 (2, 2)，偏置 `b1` 的形状是 (2, 1)。
- 第二层权重 `W2` 的形状是 (1, 2)，偏置 `b2` 的形状是 (1, 1)。

在前向传播过程中，我们首先计算第一层的激活 `A1`，然后计算第二层的激活 `A2`。这里是详细的步骤：

1. 计算 `Z1`：将输入 `X` 转置（变为 (2, 4)），然后与权重 `W1` 点积（结果为 (2, 2)），最后加上偏置 `b1`（结果不变，仍然是 (2, 2)），得到 `Z1` 的形状是 (2, 4)。
2. 应用激活函数 `sigmoid` 得到 `A1`，形状仍然是 (2, 4)。
3. 计算 `Z2`：将 `A1` 与权重 `W2` 点积（结果为 (2, 1) 与 (1, 2) 点积，结果为 (2, 1)），然后加上偏置 `b2`（形状为 (1, 1)），得到 `Z2` 的形状是 (1, 4)。
4. 应用激活函数 `sigmoid` 得到 `A2`，形状是 (1, 4)。

现在，我们已经了解了网络的结构和每一层的输出形状。接下来，我们需要确保反向传播过程中的计算与这些形状相匹配。特别是在计算 `dA1` 时，我们需要使用 `W2.T` 来考虑 `A2` 到 `A1` 的影响。由于 `W2` 的形状是 (1, 2)，`W2.T` 的形状将是 (2, 1)。然而，由于 `dZ2` 的形状是 (4, 4)，我们需要确保 `W2.T` 能够与 `dZ2` 进行点积。

为了解决这个问题，我们需要重新考虑网络结构，特别是权重 `W2` 的设计。如果 `A1` 的形状是 (2, 4) 并且我们需要将这些激活传递到一个形状为 (1, 4) 的 `A2`，那么 `W2` 应该是一个形状为 (4, 1) 的矩阵，以便能够将 (2, 4) 的 `A1` 映射到 (1, 4) 的 `A2`。这样，`W2.T` 将是 (1, 4)，可以与 (4, 4) 的 `dZ2` 进行点积。

请检查您的网络设计和权重初始化，确保它们与前向传播和反向传播的计算相匹配。如果需要进一步的帮助，请提供更多的信息。

In [41]:
import numpy as np

# 激活函数及其导数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def softmax(x):
    exp_x = np.exp(x - np.max(x))  # 避免数值溢出
    return exp_x / exp_x.sum(axis=0, keepdims=True)

def softmax_derivative(x):
    exp_x = np.exp(x - np.max(x))
    softmax_x = exp_x / exp_x.sum(axis=0, keepdims=True)
    return softmax_x * (1 - softmax_x)

# 初始化权重和偏置
def initialize_parameters(n_x, n_h, n_y):
    # 隐藏层权重 W1 的形状是（5, 2），偏置 b1 的形状是（5, 1）
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    # 输出层权重 W2 的形状是（5, 5），偏置 b2 的形状是（5, 1）
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    return W1, b1, W2, b2

# 前向传播
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = np.dot(W1, X.T) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = softmax(Z2)  # 使用 softmax 函数
    return Z1, A1, Z2, A2

# 计算损失（交叉熵损失）
def compute_loss(A2, Y):
    m = Y.shape[0]
    loss = -np.sum(Y * np.log(A2.T)) / m
    return loss

# 反向传播
def backward_propagation(X, Y, W1, W2, A1, A2, Z1, Z2):
    m = X.shape[0]
    Y = Y.reshape(-1, 4) # 由于 Y 的形状是（4, 5），需要转置为（5, 4）
    dZ2 = A2 - Y
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=0, keepdims=True)
    dA1 = np.dot(W2.T, dZ2) * sigmoid_derivative(Z1)
    dW1 = (1/m) * np.dot(dA1, X)
    db1 = (1/m) * np.sum(dA1, axis=0, keepdims=True)
    return dW1, db1, dW2, db2

# 更新参数
def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):



    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

# 训练模型
def train(X, Y, W1, b1, W2, b2, learning_rate, epochs):
    for epoch in range(epochs):
        Z1, A1, Z2, A2 = forward_propagation(X, W1, b1, W2, b2)
        loss = compute_loss(A2, Y)
        dW1, db1, dW2, db2 = backward_propagation(X, Y, W1, W2, A1, A2, Z1, Z2)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss}")

# 示例数据
X = np.array([[0,0], [0,1], [1,0], [1,1]]) # X的形状是（4,2）
Y = np.array([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0]]) # Y的形状是（4,5），one-hot 编码

# 初始化参数: 输入特征的数量为2，隐藏层神经元数量为2，输出层神经元数量为5
n_x = X.shape[1]
n_h = 2
n_y = 5
W1, b1, W2, b2 = initialize_parameters(n_x, n_h, n_y)

# 训练模型
learning_rate = 0.1
epochs = 1000
train(X, Y, W1, b1, W2, b2, learning_rate, epochs)

ValueError: non-broadcastable output operand with shape (2,1) doesn't match the broadcast shape (2,4)

In [23]:
W1 = np.random.randn(n_h, n_x) * 0.01

In [3]:
W1

array([[-0.00041989, -0.00443666],
       [-0.0009294 ,  0.00248696]])

tensor

In [42]:
import torch

# 激活函数及其导数
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# 初始化权重和偏置
def initialize_parameters(n_x, n_h, n_y):
    np.random.seed(42)  # 为了可重复性
    W1 = torch.randn(n_h, n_x) * 0.01
    b1 = torch.zeros(n_h, 1)
    W2 = torch.randn(n_y, n_h) * 0.01
    b2 = torch.zeros(n_y, 1)
    return W1, b1, W2, b2

# 前向传播
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = torch.mm(W1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = torch.mm(W2, A1) + b2
    A2 = sigmoid(Z2)
    return Z1, A1, Z2, A2

# 计算损失
def compute_loss(A2, Y):
    loss = torch.mean(-torch.sum(Y * torch.log(A2) + (1 - Y) * torch.log(1 - A2), dim=1))
    return loss

# 反向传播
def backward_propagation(X, Y, W1, W2, A1, A2, Z1, Z2):
    m = X.shape[0]
    dZ2 = A2 - Y
    dW2 = torch.mm(dZ2, A1.T) / m
    db2 = torch.sum(dZ2, dim=0, keepdim=True) / m
    dA1 = torch.mm(W2.T, dZ2) * sigmoid_derivative(Z1)
    dW1 = torch.mm(dA1, X.T) / m
    db1 = torch.sum(dA1, dim=0, keepdim=True) / m
    return dW1, db1, dW2, db2

# 更新参数
def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

# 训练模型
def train(X, Y, W1, b1, W2, b2, learning_rate, epochs):
    for epoch in range(epochs):
        Z1, A1, Z2, A2 = forward_propagation(X, W1, b1, W2, b2)
        loss = compute_loss(A2, Y)
        dW1, db1, dW2, db2 = backward_propagation(X, Y, W1, W2, A1, A2, Z1, Z2)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")

# 示例数据
X = torch.tensor([[0,0], [0,1], [1,0], [1,1]], dtype=torch.float32)
Y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# 初始化参数
n_x = X.shape[1]
n_h = 2
n_y = 1
W1, b1, W2, b2 = initialize_parameters(n_x, n_h, n_y)

# 训练模型
learning_rate = 0.1
epochs = 1000
train(X, Y, W1, b1, W2, b2, learning_rate, epochs)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x2 and 4x2)

nn.functional 
在这个改写的版本中，我们使用了以下torch.nn.functional模块的函数：

F.binary_cross_entropy：计算二元交叉熵损失，这是二分类问题的常用损失函数。
torch.sigmoid：应用Sigmoid激活函数。
请注意，backward_propagation函数中的dA1计算使用了Sigmoid函数的导数，这是因为F.binary_cross_entropy返回的梯度是关于输出的，而不是关于输入的。因此，我们需要手动计算关于输入的梯度。

使用torch.nn.functional模块可以使代码更加简洁和易于理解，同时也能够利用PyTorch的一些内部优化。

In [43]:
import torch
import torch.nn.functional as F

# 初始化权重和偏置
def initialize_parameters(n_x, n_h, n_y):
    W1 = torch.randn(n_h, n_x) * 0.01
    b1 = torch.zeros(n_h, 1)
    W2 = torch.randn(n_y, n_h) * 0.01
    b2 = torch.zeros(n_y, 1)
    return W1, b1, W2, b2

# 前向传播
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = torch.mm(W1, X) + b1
    A1 = torch.sigmoid(Z1)
    Z2 = torch.mm(W2, A1) + b2
    A2 = torch.sigmoid(Z2)
    return A1, A2

# 计算损失
def compute_loss(A2, Y):
    loss = F.binary_cross_entropy(A2, Y)
    return loss

# 反向传播
def backward_propagation(X, Y, A1, A2, W1, W2):
    m = X.shape[0]
    dA2 = A2 - Y
    dW2 = torch.mm(dA2, A1.T) / m
    dA1 = torch.mm(W2.T, dA2) * (A1 * (1 - A1))
    dW1 = torch.mm(dA1, X.T) / m
    return dW1, dA1

# 更新参数
def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

# 训练模型
def train(X, Y, W1, b1, W2, b2, learning_rate, epochs):
    for epoch in range(epochs):
        A1, A2 = forward_propagation(X, W1, b1, W2, b2)
        loss = compute_loss(A2, Y)
        dW1, dA1 = backward_propagation(X, Y, A1, A2, W1, W2)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, dA1, dW2, db1, db2, learning_rate)
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")

# 示例数据
X = torch.tensor([[0,0], [0,1], [1,0], [1,1]], dtype=torch.float32)
Y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# 初始化参数
n_x = X.shape[1]
n_h = 2
n_y = 1
W1, b1, W2, b2 = initialize_parameters(n_x, n_h, n_y)

# 训练模型
learning_rate = 0.1
epochs = 1000
train(X, Y, W1, b1, W2, b2, learning_rate, epochs)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x2 and 4x2)

nn module

在这个代码中，我们定义了一个名为SimpleNN的类，它继承自torch.nn.Module。在这个类中，我们定义了两个线性层（self.layer1和self.layer2），并在forward方法中实现了前向传播逻辑。我们还定义了一个initialize_model函数来创建模型的实例。

train函数中，我们使用了torch.optim.SGD优化器来更新模型的权重。在每次迭代中，我们首先清空梯度，然后执行前向传播，计算损失，执行反向传播，并更新模型参数。

In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 定义神经网络模型
class SimpleNN(nn.Module):
    def __init__(self, n_x, n_h, n_y):
        super(SimpleNN, self).__init__()
        self.n_h = n_h
        self.layer1 = nn.Linear(n_x, n_h)  # 输入层到隐藏层的线性变换
        self.layer2 = nn.Linear(n_h, n_y)  # 隐藏层到输出层的线性变换

    def forward(self, X):
        A1 = torch.sigmoid(self.layer1(X))  # 应用Sigmoid激活函数
        A2 = torch.sigmoid(self.layer2(A1))  # 应用Sigmoid激活函数
        return A1, A2

# 初始化模型
def initialize_model(n_x, n_h, n_y):
    model = SimpleNN(n_x, n_h, n_y)
    return model

# 计算损失
def compute_loss(A2, Y):
    loss = F.binary_cross_entropy(A2, Y)
    return loss

# 训练模型
def train(model, X, Y, learning_rate, epochs):
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  # 定义优化器
    for epoch in range(epochs):
        optimizer.zero_grad()  # 清空之前的梯度
        A1, A2 = model(X)  # 前向传播
        loss = compute_loss(A2, Y)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")

# 示例数据
X = torch.tensor([[0,0], [0,1], [1,0], [1,1]], dtype=torch.float32)
Y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# 初始化模型
n_x = X.shape[1]
n_h = 2
n_y = 1
model = initialize_model(n_x, n_h, n_y)

# 训练模型
learning_rate = 0.1
epochs = 1000
train(model, X, Y, learning_rate, epochs)

Epoch 0, Loss: 0.736793041229248
Epoch 100, Loss: 0.6935387253761292
Epoch 200, Loss: 0.6934777498245239
Epoch 300, Loss: 0.6934468746185303
Epoch 400, Loss: 0.6934185028076172
Epoch 500, Loss: 0.6933924555778503
Epoch 600, Loss: 0.6933683156967163
Epoch 700, Loss: 0.6933459639549255
Epoch 800, Loss: 0.6933250427246094
Epoch 900, Loss: 0.6933053731918335


Refactor using Dataset and DataLoader

在这个重构的代码中，我们首先定义了一个SimpleDataset类，它继承自torch.utils.data.Dataset。这个类负责存储数据（X和Y），并在__getitem__方法中返回单个样本，在__len__方法中返回数据集的大小。

然后，我们创建了一个DataLoader实例，它将为我们提供批处理的数据。DataLoader可以自动地将数据集分割成小批量，并且可以在每个epoch开始时打乱数据。

在train函数中，我们遍历dataloader来获取每个批次的数据，并执行前向传播、损失计算、反向传播和参数更新。



In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# 定义数据集类
class SimpleDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

# 定义神经网络模型
class SimpleNN(nn.Module):
    def __init__(self, n_x, n_h, n_y):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(n_x, n_h)
        self.layer2 = nn.Linear(n_h, n_y)

    def forward(self, X):
        A1 = torch.sigmoid(self.layer1(X))
        A2 = torch.sigmoid(self.layer2(A1))
        return A2

# 初始化模型
def initialize_model(n_x, n_h, n_y):
    model = SimpleNN(n_x, n_h, n_y)
    return model

# 计算损失
def compute_loss(model, X, Y):
        output = model(X)
        loss = F.binary_cross_entropy(output, Y)
        return loss

# 训练模型
def train(model, dataloader, learning_rate, epochs):
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    for epoch in range(epochs):
        for batch_idx, (X_batch, Y_batch) in enumerate(dataloader):
            optimizer.zero_grad()
            output = model(X_batch)
            loss = compute_loss(model, X_batch, Y_batch)
            loss.backward()
            optimizer.step()
            if batch_idx % 100 == 0:
                print(f"Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}")

# 示例数据
X = torch.tensor([[0,0], [0,1], [1,0], [1,1]], dtype=torch.float32)
Y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# 创建数据集
dataset = SimpleDataset(X, Y)

# 创建数据加载器
batch_size = 2  # 可以调整批量大小
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 初始化模型
n_x = X.shape[1]
n_h = 2
n_y = 1
model = initialize_model(n_x, n_h, n_y)

# 训练模型
learning_rate = 0.1
epochs = 1000
train(model, dataloader, learning_rate, epochs)

Epoch 0, Batch 0, Loss: 0.6954602599143982
Epoch 1, Batch 0, Loss: 0.7854986190795898
Epoch 2, Batch 0, Loss: 0.613932728767395
Epoch 3, Batch 0, Loss: 0.6937421560287476
Epoch 4, Batch 0, Loss: 0.6267933249473572
Epoch 5, Batch 0, Loss: 0.6976216435432434
Epoch 6, Batch 0, Loss: 0.7042776346206665
Epoch 7, Batch 0, Loss: 0.6854599714279175
Epoch 8, Batch 0, Loss: 0.6852806806564331
Epoch 9, Batch 0, Loss: 0.6920747756958008
Epoch 10, Batch 0, Loss: 0.696416974067688
Epoch 11, Batch 0, Loss: 0.6849043369293213
Epoch 12, Batch 0, Loss: 0.7288597226142883
Epoch 13, Batch 0, Loss: 0.6602154970169067
Epoch 14, Batch 0, Loss: 0.7231813669204712
Epoch 15, Batch 0, Loss: 0.7223381996154785
Epoch 16, Batch 0, Loss: 0.6958688497543335
Epoch 17, Batch 0, Loss: 0.6957916617393494
Epoch 18, Batch 0, Loss: 0.6957243084907532
Epoch 19, Batch 0, Loss: 0.6914406418800354
Epoch 20, Batch 0, Loss: 0.7024766206741333
Epoch 21, Batch 0, Loss: 0.6913979649543762
Epoch 22, Batch 0, Loss: 0.6913831233978271


using nn.parameter

在这个重构的代码中，SimpleNN类中的_initialize_weights方法使用nn.init模块来初始化权重和偏置。nn.init.xavier_uniform_是一个常用的权重初始化方法，它适用于激活函数是Sigmoid或Tanh的情况。nn.init.zeros_用于将偏置初始化为零。

请注意，我们没有显式地将self.linear1.weight和self.linear1.bias注册为nn.Parameter，因为在PyTorch中，nn.Linear的权重和偏置默认就是nn.Parameter。这意味着它们会自动被优化器跟踪和更新。



In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# 定义数据集类
class SimpleDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

# 定义神经网络模型
class SimpleNN(nn.Module):
    def __init__(self, n_x, n_h, n_y):
        super(SimpleNN, self).__init__()
        self.n_h = n_h
        self.linear1 = nn.Linear(n_x, n_h)
        self.linear2 = nn.Linear(n_h, n_y)
        # 初始化权重和偏置
        self._initialize_weights()

    def forward(self, X):
        A1 = torch.sigmoid(self.linear1(X))
        A2 = torch.sigmoid(self.linear2(A1))
        return A2

    def _initialize_weights(self):
        # 使用nn.init来初始化权重和偏置
        nn.init.xavier_uniform_(self.linear1.weight)
        nn.init.zeros_(self.linear1.bias)
        nn.init.xavier_uniform_(self.linear2.weight)
        nn.init.zeros_(self.linear2.bias)

# 初始化模型
def initialize_model(n_x, n_h, n_y):
    model = SimpleNN(n_x, n_h, n_y)
    return model

# 计算损失
def compute_loss(output, Y):
    loss = F.binary_cross_entropy(output, Y)
    return loss

# 训练模型
def train(model, dataloader, learning_rate, epochs):
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    for epoch in range(epochs):
        for X_batch, Y_batch in dataloader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = compute_loss(output, Y_batch)
            loss.backward()
            optimizer.step()

# 示例数据
X = torch.tensor([[0,0], [0,1], [1,0], [1,1]], dtype=torch.float32)
Y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# 创建数据集
dataset = SimpleDataset(X, Y)

# 创建数据加载器
batch_size = 2  # 可以调整批量大小
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 初始化模型
n_x = X.shape[1]
n_h = 2
n_y = 1
model = initialize_model(n_x, n_h, n_y)

# 训练模型
learning_rate = 0.1
epochs = 1000
train(model, dataloader, learning_rate, epochs)

add validation 

在这个示例中，我们首先使用random_split函数将原始数据集随机分割为训练集和验证集。然后，我们创建了两个DataLoader：一个用于训练，另一个用于验证。

在训练循环中，我们首先将模型设置为训练模式，然后执行前向传播、损失计算、反向传播和参数更新。在每个epoch结束时，我们将模型设置为评估模式，并计算验证集上的损失和准确率。注意，在评估模式下，我们使用torch.no_grad()上下文管理器来禁用梯度计算，这样可以节省内存并防止影响模型评估。

In [47]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

# 定义数据集类
class SimpleDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

# 定义神经网络模型
class SimpleNN(nn.Module):
    def __init__(self, n_x, n_h, n_y):
        super(SimpleNN, self).__init__()
        self.n_h = n_h
        self.linear1 = nn.Linear(n_x, n_h)
        self.linear2 = nn.Linear(n_h, n_y)

    def forward(self, X):
        A1 = torch.sigmoid(self.linear1(X))
        A2 = torch.sigmoid(self.linear2(A1))
        return A2

# 初始化模型
def initialize_model(n_x, n_h, n_y):
    model = SimpleNN(n_x, n_h, n_y)
    model.apply(initialize_weights)  # 应用权重初始化
    return model

# 初始化权重的函数
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

# 计算损失
def compute_loss(output, Y):
    loss = F.binary_cross_entropy(output, Y)
    return loss

# 训练模型
def train(model, train_loader, val_loader, learning_rate, epochs):
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    for epoch in range(epochs):
        model.train()  # 设置模型为训练模式
        for X_batch, Y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = compute_loss(output, Y_batch)
            loss.backward()
            optimizer.step()

        # 在每个epoch结束时进行验证
        model.eval()  # 设置模型为评估模式
        val_loss = 0
        correct_predictions = 0
        with torch.no_grad():  # 在验证阶段不计算梯度
            for X_batch, Y_batch in val_loader:
                output = model(X_batch)
                val_loss += compute_loss(output, Y_batch).item()
                _, predictions = torch.max(output.data, 1)
                correct_predictions += (predictions == Y_batch).sum().item()

        val_loss /= len(val_loader.dataset)
        accuracy = correct_predictions / len(val_loader.dataset)
        print(f"Epoch {epoch}: Training Loss {val_loss}, Validation Loss {val_loss}, Validation Accuracy {accuracy:.4f}")

# 示例数据
X = torch.tensor([[0,0], [0,1], [1,0], [1,1]], dtype=torch.float32)
Y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# 创建数据集
dataset = SimpleDataset(X, Y)

# 创建数据加载器
batch_size = 2  # 可以调整批量大小
train_size = int(0.8 * len(dataset))  # 80%的数据用于训练
val_size = len(dataset) - train_size  # 剩余20%的数据用于验证
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# 初始化模型
n_x = X.shape[1]
n_h = 2
n_y = 1
model = initialize_model(n_x, n_h, n_y)

# 训练模型
learning_rate = 0.1
epochs = 1000
train(model, train_loader, val_loader, learning_rate, epochs)

Epoch 0: Training Loss 0.6307094097137451, Validation Loss 0.6307094097137451, Validation Accuracy 0.0000
Epoch 1: Training Loss 0.6739773750305176, Validation Loss 0.6739773750305176, Validation Accuracy 0.0000
Epoch 2: Training Loss 0.7157135009765625, Validation Loss 0.7157135009765625, Validation Accuracy 0.0000
Epoch 3: Training Loss 0.7572778463363647, Validation Loss 0.7572778463363647, Validation Accuracy 0.0000
Epoch 4: Training Loss 0.750220000743866, Validation Loss 0.750220000743866, Validation Accuracy 0.0000
Epoch 5: Training Loss 0.7902514934539795, Validation Loss 0.7902514934539795, Validation Accuracy 0.0000
Epoch 6: Training Loss 0.8283535242080688, Validation Loss 0.8283535242080688, Validation Accuracy 0.0000
Epoch 7: Training Loss 0.8154942393302917, Validation Loss 0.8154942393302917, Validation Accuracy 0.0000
Epoch 8: Training Loss 0.8037827014923096, Validation Loss 0.8037827014923096, Validation Accuracy 0.0000
Epoch 9: Training Loss 0.7931150794029236, Valid

创建两个函数：fit()用于训练模型，get_data()用于加载和预处理数据

现在，我们有了两个独立的函数：get_data()和fit()。get_data()函数接受原始数据集的特征矩阵X和标签向量Y，然后创建并返回训练和验证数据加载器。fit()函数接受模型实例、训练和验证数据加载器、学习率和训练轮数，然后执行训练和验证过程。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

# 定义数据集类
class SimpleDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

# 定义神经网络模型
class SimpleNN(nn.Module):
    def __init__(self, n_x, n_h, n_y):
        super(SimpleNN, self).__init__()
        self.n_h = n_h
        self.linear1 = nn.Linear(n_x, n_h)
        self.linear2 = nn.Linear(n_h, n_y)

    def forward(self, X):
        A1 = torch.sigmoid(self.linear1(X))
        A2 = torch.sigmoid(self.linear2(A1))
        return A2

# 初始化权重的函数
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

# 计算损失
def compute_loss(output, Y):
    loss = F.binary_cross_entropy(output, Y)
    return loss

# 加载和预处理数据
def get_data(X, Y):
    dataset = SimpleDataset(X, Y)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    batch_size = 2
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    return train_loader, val_loader

# 训练模型
def fit(model, train_loader, val_loader, learning_rate, epochs):
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    for epoch in range(epochs):
        model.train()  # 设置模型为训练模式
        for X_batch, Y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = compute_loss(output, Y_batch)
            loss.backward()
            optimizer.step()

        # 在每个epoch结束时进行验证
        model.eval()  # 设置模型为评估模式
        val_loss = 0
        correct_predictions = 0
        with torch.no_grad():  # 在验证阶段不计算梯度
            for X_batch, Y_batch in val_loader:
                output = model(X_batch)
                val_loss += compute_loss(output, Y_batch).item()
                _, predictions = torch.max(output.data, 1)
                correct_predictions += (predictions == Y_batch).sum().item()

        val_loss /= len(val_loader.dataset)
        accuracy = correct_predictions / len(val_loader.dataset)
        print(f"Epoch {epoch}: Training Loss {val_loss}, Validation Loss {val_loss}, Validation Accuracy {accuracy:.4f}")

# 示例数据
X = torch.tensor([[0,0], [0,1], [1,0], [1,1]], dtype=torch.float32)
Y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# 初始化模型
n_x = X.shape[1]
n_h = 2
n_y = 1
model = SimpleNN(n_x, n_h, n_y)
model.apply(initialize_weights)

# 加载数据
train_loader, val_loader = get_data(X, Y)

# 训练模型
learning_rate = 0.1
epochs = 1000
fit(model, train_loader, val_loader, learning_rate, epochs)